## 1. boxes will have labels 'A1', 'A2', ..., 'I9'
## 2. rows, columns, and 3x3 squares, will be called units
## 3. each unit is a set of 9 boxes
## 4. there are 27 units in total
## 5. For a particular box (such as 'A1'), its peers will be all other boxes that belong to the same unit (namely, those that belong to the same row, column, or 3x3 square).


In [1]:
row = 'ABCDEFGHI'
col = '123456789'

def cross(a,b):
    return [r+c for r in a for c in b]

boxes = cross(row,col)
row_unit = [cross(r,col) for r in row]
col_unit = [cross(row,c) for c in col]
square_unit = [cross(r,c) for r in ['ABC','DEF','GFI'] for c in ['123','456','789'] ]
unitlist = row_unit + col_unit + square_unit


#print (cross(row,col))
print (row_unit)
print (col_unit)
print (square_unit)
#print (unitlist)

[['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'], ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9'], ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'], ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'], ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9'], ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9'], ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9'], ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9'], ['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9']]
[['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'], ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'], ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3'], ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4'], ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'I5'], ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'I6'], ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'I7'], ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'I8'], ['A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9

In [2]:
units = dict((s, [u for u in unitlist if s in u]) 
             for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s]))
             for s in boxes)

In [3]:
# Dictionary Form
def grid_values(grid):
    chars = []
    digits = '123456789'
    for c in grid:
        if c in digits:
            chars.append(c)
        if c == '.':
            chars.append(digits)
    assert len(chars) == 81
    return dict(zip(boxes, chars))

In [4]:
a = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')

In [5]:
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in row:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in col))
        if r in 'CF': print(line)
    return

In [12]:
# Elimination
#Now, let's finish the code for the function eliminate, which will take as input a puzzle in dictionary form. 
#The function will iterate over all the boxes in the puzzle that only have one value assigned to them, 
#and it will remove this value from every one of its peers.


def eliminate(values):   #values is the dictionary form of the sudoku
    solved_values = [box for box in values.keys() if len(values[box]) == 1]   #creat a list of keys which only has one number
    for box in solved_values:   #loop through the boxes with only one value
        digit = values[box]   #name digit as the one value of those boxes
        for peer in peers[box]:   #loop through the peer of the boxes with only one value
            values[peer] = values[peer].replace(digit,'')   #within the possibilities of each peer, replace the digit with blank   
    return values


In [13]:
#Only Choice
#take as input a puzzle in dictionary form. The function will go through all the units, 
#and if there is a unit with a digit that only fits in one possible box, it will assign that digit to that box.

def only_choice(values):
    new_values = values.copy()  # note: do not modify original values
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                new_values[dplaces[0]] = digit
    return new_values




In [14]:
##Constraint Propagation

a = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')

def reduce_puzzle(values):

    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    
    stalled = False
    while not stalled:   #while not false, do the following....
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        # Use the Eliminate Strategy
        values = eliminate(values)
        # Use the Only Choice Strategy
        values = only_choice(values)
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after == 81
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values



In [7]:
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
values = grid_values(grid2)

In [ ]:
display(reduce_puzzle(values))